In [ ]:
from experiments.datasets import DATASETS, Solution
from distribution_optimization_py.problem import ScaledGaussianMixtureProblem
import random 
import numpy as np
from cma import fmin

In [ ]:
config = {
    "popsize": 26,
    "sigma0": 5.33360647287386,
    "incpopsize": 1,
    "tolfun": 5.6629044577000325e-09
}

In [ ]:
dataset_to_solutions = {}
for dataset in DATASETS:
    dataset_to_solutions[dataset.name] = {}
    for budget in [1000, 2000, 5000, 10000, 15000]:
        dataset_to_solutions[dataset.name][budget] = []
        for seed in range(1, 31):
            np.random.seed(seed)
            random.seed(seed)
            problem = ScaledGaussianMixtureProblem(dataset.data, dataset.nr_of_modes)
            start = problem.initialize_warm_start()
            options = {
                "bounds": [problem.lower, problem.upper],
                "verbose": -9,
                "maxfevals": budget,
                "popsize": config["popsize"],
                "tolfun": config["tolfun"],
                "seed": seed
            }
            res = fmin(
                problem,
                start,
                config["sigma0"],
                options=options,
                bipop=True,
                restart_from_best=True,
                restarts=10,
                incpopsize=config["incpopsize"],
            )
            genome, fitness = res[0], res[1]
            scaled_genome = problem.reals_to_internal(genome)
            solution = Solution(
                fitness,
                genome,
                scaled_genome,
            )
            dataset_to_solutions[dataset.name][budget].append(solution)

In [ ]:
for dataset_name in dataset_to_solutions:
    for budget in dataset_to_solutions[dataset_name]:
        solutions = np.array(
            [x.scaled_genome for x in dataset_to_solutions[dataset_name][budget]]
        )
        np.savetxt(f"results_cma_es_bipop/{dataset_name}_solutions_{budget}.csv", solutions, delimiter=",")